In [73]:
# Lab 2- part 1
#make necesarry imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import correlation, cosine
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.metrics import pairwise_distances
from sklearn.metrics import mean_squared_error
from math import sqrt
import sys, os
from contextlib import contextmanager

In [42]:
#M is user-item ratings matrix where ratings are integers from 1-10
M = np.asarray([[3,7,4,9,9,7], [7,0,5,3,8,8],
               [7,5,5,0,8,4],
               [5,6,8,5,9,8],
               [5,8,8,8,10,9],
               [7,7,0,4,7,8]])
M=pd.DataFrame(M)

#declaring k,metric as global which can be changed by the user later
global k,metric
k=1
metric='cosine' #can be changed to 'correlation' for Pearson correlation similaries

In [43]:
M

,0,1,2,3,4,5
0,3,7,4,9,9,7
1,7,0,5,3,8,8
2,7,5,5,0,8,4
3,5,6,8,5,9,8
4,5,8,8,8,10,9
5,7,7,0,4,7,8


In [6]:
#get cosine similarities for ratings matrix M; pairwise_distances returns the distances between ratings and hence
#similarities are obtained by subtracting distances from 1
cosine_sim = 1-pairwise_distances(M, metric="cosine")

In [11]:
#Cosine similarity matrix
pd.DataFrame(cosine_sim)

,0,1,2,3,4,5
0,1.000000,0.799268,0.779227,0.934622,0.973890,0.884600
1,0.799268,1.000000,0.874744,0.905850,0.866146,0.827036
2,0.779227,0.874744,1.000000,0.909513,0.865454,0.853275
3,0.934622,0.905850,0.909513,1.000000,0.989344,0.865614
4,0.973890,0.866146,0.865454,0.989344,1.000000,0.881640
5,0.884600,0.827036,0.853275,0.865614,0.881640,1.000000


In [12]:
#get pearson similarities for ratings matrix M
pearson_sim = 1-pairwise_distances(M, metric="correlation")

In [14]:
#Pearson correlation similarity matrix
pd.DataFrame(pearson_sim)

,0,1,2,3,4,5
0,1.000000,-0.137446,-0.357398,0.208179,0.761905,0.277350
1,-0.137446,1.000000,0.453897,0.515910,0.112456,0.218328
2,-0.357398,0.453897,1.000000,0.451378,-0.042888,0.297373
3,0.208179,0.515910,0.451378,1.000000,0.763325,-0.057739
4,0.761905,0.112456,-0.042888,0.763325,1.000000,0.039621
5,0.277350,0.218328,0.297373,-0.057739,0.039621,1.000000


In [44]:
#This function finds k similar users given the user_id and ratings matrix M
#Note that the similarities are same as obtained via using pairwise_distances
def findksimilarusers(user_id, ratings, metric = metric, k=k):
    similarities=[]
    indices=[]
    model_knn = NearestNeighbors(metric = metric, algorithm = 'brute') 
    model_knn.fit(ratings)

    distances, indices = model_knn.kneighbors(ratings.iloc[user_id-1, :].values.reshape(1, -1), n_neighbors = k+1)
    similarities = 1-distances.flatten()
    print('{0} most similar users for User {1}:\n'.format(k,user_id))
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i]+1 == user_id:
            continue;

        else:
            print('{0}: User {1}, with similarity of {2}'.format(i, indices.flatten()[i]+1, similarities.flatten()[i]))
            
    return similarities,indices

In [45]:
similarities,indices = findksimilarusers(3,M, metric='cosine')

1 most similar users for User 3:

1: User 4, with similarity of 0.909512689340191


In [71]:
#This function predicts rating for specified user-item combination based on user-based approach
def predict_userbased(user_id, item_id, ratings, metric = metric, k=k):
    prediction=0
    similarities, indices=findksimilarusers(user_id, ratings,metric, k) #similar users based on cosine similarity
    mean_rating = ratings.loc[user_id-1,:].mean() #to adjust for zero based indexing
    sum_wt = np.sum(similarities)-1
    product=1
    wtd_sum = 0 
    
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i]+1 == user_id:
            continue;
        else: 
            ratings_diff = ratings.iloc[indices.flatten()[i],item_id-1]-np.mean(ratings.iloc[indices.flatten()[i],:])
            product = ratings_diff * (similarities[i])
            wtd_sum = wtd_sum + product

    
    prediction = int(round(mean_rating + (wtd_sum/sum_wt)))
    print('\nPredicted rating for user {0} -> item {1}: {2}'.format(user_id,item_id,prediction))

    return prediction

In [72]:
predict_userbased(3,4,M);

1 most similar users for User 3:

1: User 4, with similarity of 0.909512689340191

Predicted rating for user 3 -> item 4: 3


In [80]:
# Lab 2- part 2
import nltk
sent1 = "The school art."
sent2 = "Schooling Kids"
sent3 = "Closing Schools"
sent4 = "military school" # This has the same words as sent1 with a different order.
sent5 = "This is a military school"
sent6 = "School Book"
 
ed_sent_4_2 = nltk.edit_distance(sent4, sent2)
ed_sent_4_3 = nltk.edit_distance(sent4, sent3)
ed_sent_4_5 = nltk.edit_distance(sent4, sent5)
ed_sent_4_6 = nltk.edit_distance(sent4, sent6)

print(ed_sent_4_2, 'Edit Distance between sent4 and sent2')
print(ed_sent_4_3, 'Edit Distance between sent4 and sent3')
print(ed_sent_4_5, 'Edit Distance between sent4 and sent5')
print(ed_sent_4_6, 'Edit Distance between sent4 and sent6')

15 Edit Distance between sent4 and sent2
9 Edit Distance between sent4 and sent3
10 Edit Distance between sent4 and sent5
12 Edit Distance between sent4 and sent6
